In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
import time, re

# Set up Firefox options to configure the browser window size
firefox_options = Options()
firefox_options.add_argument("--width=1200")  # Set the width of the browser window
firefox_options.add_argument("--height=1000") # Set the height of the browser window

# Initialize the Firefox driver with the specified options
# driver = webdriver.Firefox(options=firefox_options)


In [ ]:
import re

def parse_and_calculate(input_text):
    """
    Parses the given input text to perform arithmetic operations or handle specific queries.

    Parameters:
    input_text (str): The input string containing either an arithmetic operation in words or a specific query about bags and stones.

    Returns:
    str: The result of the calculation or an error message if the input is invalid.
    """

    # Handle the special case with the specific pattern for bags and stones
    special_case_pattern = re.compile(r"How many stones are there when there are (\d+) bags with (\d+) stones packed\?")
    match = special_case_pattern.match(input_text)
    
    if match:
        # Extract the numbers of bags and stones from the matched pattern
        bags = int(match.group(1))
        stones_per_bag = int(match.group(2))
        # Calculate the total number of stones
        result = bags * stones_per_bag
        return f"Result: {result}"
    
    # Dictionary to map number words to their corresponding integers
    words_to_numbers = {"zero": 0,
        "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6,
        "seven": 7, "eight": 8, "nine": 9, "ten": 10, "eleven": 11,
        "twelve": 12, "thirteen": 13, "fourteen": 14, "fifteen": 15,
        "sixteen": 16, "seventeen": 17, "eighteen": 18, "nineteen": 19,
        "twenty": 20, "twenty-one": 21, "twenty-two": 22, "twenty-three": 23,
        "twenty-four": 24, "twenty-five": 25, "twenty-six": 26, "twenty-seven": 27,
        "twenty-eight": 28, "twenty-nine": 29, "thirty": 30
    }

    # Split the input text into tokens to identify numbers and operations
    tokens = input_text.split()
    
    # Ensure the input has the correct format: two numbers and an operation
    if len(tokens) == 3:
        # Retrieve the corresponding integers from the dictionary
        num1 = words_to_numbers.get(tokens[0].lower())
        operation = tokens[1]
        num2 = words_to_numbers.get(tokens[2].lower())
        
        # Perform the appropriate arithmetic operation
        if num1 is not None and num2 is not None:
            if operation == '+':
                result = num1 + num2
            elif operation == '-':
                result = num1 - num2
            elif operation == 'x':
                result = num1 * num2
            elif operation == '/':
                # Handle division by zero
                if num2 == 0:
                    return "Input rejected: Division by zero."
                result = num1 / num2
            else:
                return "Invalid operation"
            
            # Return the result of the calculation
            return f"Result: {result}"
    
    # Return an error message if the input is not in a valid format
    return "Invalid input"

def test_parse_and_calculate():
    """
    Tests the parse_and_calculate function with various inputs.
    """
    # Test cases for arithmetic operations
    print(parse_and_calculate("sixteen - thirteen"))  # Expected output: Result: 3
    print(parse_and_calculate("twenty x four"))       # Expected output: Result: 80
    print(parse_and_calculate("thirty + five"))       # Expected output: Invalid input (since 30 + 5 is not supported)
    print(parse_and_calculate("fourteen / zero"))     # Expected output: Input rejected: Division by zero.
    
    # Test case for the special query
    print(parse_and_calculate("How many stones are there when there are 10 bags with 10 stones packed?"))  # Expected output: Result: 100
    
    # Test case for invalid input
    print(parse_and_calculate("how are you"))  # Expected output: Invalid input (as the input format is incorrect)

# Run the test cases
test_parse_and_calculate()


In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

def navigate_to_page(driver):
    """
    Navigate to the initial page and click the link to navigate to the form page.
    """
    # Navigate to the initial page
    driver.get("https://solobs.nict.go.jp/radio/cgi-bin/MainDisplay.pl")
    print("Navigated to the initial page")

    # Wait for the link to be present
    link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[@href="https://solobs.nict.go.jp/radio/cgi-bin/MainDisplay.pl"]'))
    )

    # Scroll the link into view if needed
    driver.execute_script("arguments[0].scrollIntoView(true);", link)

    # Click the link using JavaScript
    driver.execute_script("arguments[0].click();", link)
    print("Clicked the link to the form page")

    # Wait for the form to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "select_ymd")))
    print("Form loaded.")

def fill_form(driver, date, desired_hour, desired_duration):
    """
    Fill out the form with the given date, hour, and duration.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    date (str): The date to input in 'YYYY/MM/DD' format.
    desired_hour (int): The hour to select.
    desired_duration (int): The duration to select.
    """
    print(f"Entering date: {date}")

    # Use JavaScript to set the value of the date input field
    script = f'document.getElementById("select_ymd").value = "{date}";'
    driver.execute_script(script)

    # Update hidden date fields
    year, month, day = date.split('/')
    driver.execute_script(f"document.getElementById('select_year').value = '{year}';")
    driver.execute_script(f"document.getElementById('select_month').value = '{month}';")
    driver.execute_script(f"document.getElementById('select_day').value = '{day}';")

    # Locate and set the hour dropdown
    hour_dropdown = Select(driver.find_element(By.ID, 'select_hour'))
    hour_dropdown.select_by_value(str(desired_hour))

    # Locate and set the duration dropdown
    duration_dropdown = Select(driver.find_element(By.ID, 'select_span_hour'))
    duration_dropdown.select_by_value(str(desired_duration))

    # Small delay to allow any JavaScript to run
    time.sleep(1)

    # Locate the submit button by its id and click it using JavaScript
    submit_button = driver.find_element(By.ID, 'set_btn')
    driver.execute_script("arguments[0].click();", submit_button)
    
    # Wait for the new form or section with checkboxes to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'fits_table')))  # Replace 'checkbox_section_id' with the actual ID of the new form or section
    print("New form with checkboxes loaded.")

    # Wait for the table to appear after form submission
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        print("Form submitted successfully, table loaded")
    except:
        print("Form submission might have failed, table not found")
        
def retrieve_download_count(driver):
    """
    Retrieve the total number of entries available for download.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    
    Returns:
    int: The total number of entries if found, None otherwise.
    """
    try:
        # Locate the div by its ID and wait until it's present
        data_info_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "fits_table_info"))
        )

        # Extract the text from the div
        data_info_text = data_info_div.text
        print(f"Data info text: {data_info_text}")

        # Use regular expression to find the total number of entries
        match = re.search(r'of (\d+) entries', data_info_text)
        if match:
            total_entries = int(match.group(1))
            print(f"Total number of entries: {total_entries}")
            return total_entries
        else:
            print("Failed to extract the total number of entries.")
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def set_entries_to_100(driver):
    """
    Set the number of entries per page to 100.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    """
    try:
        # Wait until the 'dataTables_empty' class is no longer present
        WebDriverWait(driver, 10).until_not(
            EC.presence_of_element_located((By.CLASS_NAME, "dataTables_empty"))
        )

        # Wait until the select element is present
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "fits_table_length"))
        )

        # Use JavaScript to change the value of the dropdown to '100'
        driver.execute_script("arguments[0].value = '100';", select_element)
        
        # Optionally, trigger the change event if needed by the page's JavaScript
        driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", select_element)

        print("Set the entries per page to 100 using JavaScript.")

    except Exception as e:
        print(f"An error occurred: {e}")

def checkbox_id_generator(n, batch_size=5):
    """
    Generate batches of checkbox IDs.
    
    Parameters:
    n (int): The total number of checkboxes.
    batch_size (int): The size of each batch.
    
    Yields:
    list: A list of checkbox IDs for each batch.
    """
    for i in range(0, n, batch_size):
        yield [f'check_{j}' for j in range(i, min(i + batch_size, n))]

def check_boxes(driver, generator, flag):
    """
    Check checkboxes based on the IDs provided by the generator.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    generator (generator): The generator yielding batches of checkbox IDs.
    flag (int): A flag to indicate when all checkboxes have been checked.
    """
    try:
        # Get the next batch of checkbox IDs from the generator
        checkbox_ids = next(generator)
        print(f"Checking boxes: {checkbox_ids}")
        # Loop through the IDs provided by the generator
        for checkbox_id in checkbox_ids:
            # Wait for the checkbox to be present
            checkbox = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, checkbox_id))
            )
            
            # Scroll into view and click the checkbox
            driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked checkbox with ID: {checkbox_id}")

    except StopIteration:
        flag = 1
        print("All checkboxes have been checked.")
    except Exception as e:
        print(f"An error occurred: {e}")

def wait_for_table_update(driver):
    """
    Wait until the table is updated with new data.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    """
    # Wait until the 'dataTables_empty' class is no longer present
    WebDriverWait(driver, 10).until_not(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTables_empty"))
    )
    print("Table updated with data.")

def extract_expression(driver):
    """
    Extract the expression from a div element.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    
    Returns:
    str: The extracted expression from the div element.
    """
    try:
        # Locate the div element by its ID
        div_element = driver.find_element(By.ID, 'question_parts')
        
        # Extract the text content from the div
        expression = div_element.text.strip()
        
        return expression
    except Exception as e:
        return f"An error occurred: {e}"

def click_download_button(driver):
    """
    Click the download button.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    """
    try:
        # Wait until the button is present
        download_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'download'))
        )
        
        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", download_button)
        print("Download button clicked.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

def fill_captcha_field(driver, answer):
    """
    Fill the captcha field with the provided answer.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    answer (str): The answer to fill in the captcha field.
    """
    try:
        # Wait for the input field to be present
        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'user_answer'))
        )
        
        # Clear any existing text and input the answer
        input_field.clear()
        input_field.send_keys(answer)
        
        # Optionally, click submit if needed
        submit_button = driver.find_element(By.ID, 'download')
        driver.execute_script("arguments[0].click();", submit_button)
        
        print("Captcha field filled and submitted.")
    except Exception as e:
        print(f"An error occurred: {e}")

def wait_for_download(driver):
    """
    Wait for the download to complete by checking for the presence of the download link.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    """
    try:
        # Wait for the download link to appear
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.LINK_TEXT, 'Download'))
        )
        print("Download link found.")
    except Exception as e:
        print(f"An error occurred: {e}")

def uncheck_all_boxes_in_table(driver, table_id='fits_table'):
    """
    Uncheck all checkboxes in the specified table.

    Args:
        driver (webdriver): The Selenium WebDriver instance.
        table_id (str): The ID of the table containing checkboxes. Default is 'fits_table'.
    """
    try:
        # Wait for the table to be present
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, table_id))
        )
        
        # Find all checkboxes within the specified table
        checkboxes = table.find_elements(By.XPATH, ".//input[@type='checkbox']")
        
        # Iterate through all found checkboxes
        for checkbox in checkboxes:
            if checkbox.is_selected():  # Check if the checkbox is checked
                driver.execute_script("arguments[0].click();", checkbox)  # Uncheck using JavaScript
                print(f"Unchecked checkbox with id: {checkbox.get_attribute('id')}")

        print("All checkboxes in the table have been unchecked.")

    except Exception as e:
        print(f"An error occurred while unchecking the boxes: {e}")

def parse_result(output):
    """
    Parse the result from a string formatted as "Result: {result}".

    Args:
        output (str): The output string.

    Returns:
        int: The result parsed as an integer.

    Raises:
        ValueError: If the output format is not as expected.
    """
    # Assuming the output string format is always "Result: {result}"
    if output.startswith("Result: "):
        result = output[8:]
        return int(float(result))
    else:
        raise ValueError("Unexpected output format")

def accept_alert(driver):
    """
    Accept any present alert in the browser.

    Args:
        driver (webdriver): The Selenium WebDriver instance.
    """
    while True:
        try:
            # Wait for a short period before checking again
            time.sleep(1)
            
            # Check if the alert is present
            alert = driver.switch_to.alert
            # If an alert is found, accept it
            print(f"Alert text: {alert.text}")
            alert.accept()
            print("Alert accepted.")
            break  # Exit the loop once the alert is handled
            
        except NoAlertPresentException:
            # Continue looping if no alert is present
            continue
        except Exception as e:
            print(f"An error occurred while handling the alert: {e}")
            break  # Exit the loop on any other exception


In [12]:
# Configuration Variables
DATE = "2023/03/28"
DESIRED_HOUR = 19
DESIRED_DURATION = 16
ENTRIES_PER_PAGE = 100

In [ ]:
# Configure Firefox options
firefox_options = Options()
# Example options
# firefox_options.add_argument("--headless")  # Uncomment if you want to run in headless mode

# Initialize the Firefox WebDriver with the configured options
driver = webdriver.Firefox(options=firefox_options)

# Use the functions with variables
navigate_to_page(driver)
fill_form(driver, DATE, DESIRED_HOUR, DESIRED_DURATION)
wait_for_table_update(driver)
MAX_DOWNLOADS = retrieve_download_count(driver)

# Create a generator for the checkbox IDs with a batch size of 5
generator = checkbox_id_generator(n=MAX_DOWNLOADS, batch_size=5)

# Set the number of entries per page to the configured value
set_entries_to_100(driver)

In [ ]:
# Create a generator for the checkbox IDs with a batch size of 5
generator = checkbox_id_generator(n=MAX_DOWNLOADS, batch_size=5)
flag = 0

# Loop to perform the operations
for i in range(MAX_DOWNLOADS // 5 + 1):
    print(f"Iteration {i + 1} of {MAX_DOWNLOADS // 5 + 1}")
    
    # Navigate to the page and fill out the form
    navigate_to_page(driver)
    fill_form(driver, DATE, DESIRED_HOUR, DESIRED_DURATION)
    wait_for_table_update(driver)
    
    # Update the maximum downloads and set entries to 100
    max_downloads = retrieve_download_count(driver)
    set_entries_to_100(driver)
    
    # Uncheck all boxes and check boxes in batches
    uncheck_all_boxes_in_table(driver)
    check_boxes(driver, generator, flag)
    
    # Extract captcha expression, calculate the answer, and fill in the captcha field
    captcha = extract_expression(driver)
    answer = parse_and_calculate(captcha)
    print(f"Captcha Answer: {answer}")
    fill_captcha_field(driver, int(parse_result(answer)))
    
    # Click the download button and handle any alerts
    click_download_button(driver)
    accept_alert(driver)
    
    # Sleep for a few seconds to ensure the download starts
    time.sleep(3)
    
driver.close()